# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests, json
import gmaps
import os

# Import API key
from config import key

In [2]:
# Load city data into DataFrame from WeatherPy
city_data = "../WeatherPy/Output/city_data.csv"
city_data_df = pd.read_csv(city_data, index_col=[0], encoding="ISO-8859-1")
city_data_df.head()
# city_data_df.dtypes

,City_Name,City_Lat,City_Long,Temperature,Humidity,Cloudiness,Wind_Speed,Country,Date
0,MahÃ©bourg,-20.4081,57.7000,70.09,94,75,9.22,MU,2021-06-17 17:59:38
1,Oranjestad,12.5240,-70.0270,89.55,62,20,21.85,AW,2021-06-17 17:59:39
2,Grand Gaube,-20.0064,57.6608,70.38,84,14,1.99,MU,2021-06-17 17:59:39
3,Hasaki,35.7333,140.8333,62.98,80,10,11.34,JP,2021-06-17 17:51:44
4,Puerto Ayora,-0.7393,-90.3518,78.55,83,75,5.99,EC,2021-06-17 17:51:45


In [3]:
# Rename columns, drop NA values & create new dataframe, compare size before/after, export csv
city_data_df = city_data_df.rename(columns={'City_Name': 'City', 'City_Lat': 'Latitude', 
                                            'City_Long': 'Longitude', 'Wind_Speed': 'Wind Speed'})

city_df_clean = city_data_df.dropna(axis = 0, how ='any')

print("Old data frame length:", len(city_data_df), "\nNew data frame length:", 
       len(city_df_clean), "\nNumber of rows with at least 1 NA value: ",
       (len(city_data_df)-len(city_df_clean)))
city_df_clean.to_csv("./output_data/city_df_clean.csv")

Old data frame length: 645 
New data frame length: 639 
Number of rows with at least 1 NA value:  6


### Humidity Heatmap

In [4]:
# Configure gmaps, use Lat & Lng as locations and Humidity as weight, set Humidity dfype = float
gmaps.configure(api_key=key)
locations = city_df_clean[["Latitude", "Longitude"]]
humidity = city_df_clean["Humidity"].astype(float)

In [5]:
# Create heatmap layer and add to map
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Ideal Weather Conditions

In [6]:
# Create new dataframe that fits indicated weather criteria
ideal_cities = city_df_clean[(city_df_clean['Temperature']>80) & (city_df_clean['Temperature']<90) 
                   & (city_df_clean['Humidity']>40) & (city_df_clean['Humidity']<70) 
                   & (city_df_clean['Wind Speed']<10) & (city_df_clean['Cloudiness']<20)]
len(ideal_cities)    # check number of cities to meet rubric criteria of 10

12

### Find Nearest Hotel

In [7]:
# Create new dataframe by method drop, add empty hotel column to store names, display results
hotels_df = ideal_cities.drop(['Temperature', 'Humidity', 'Cloudiness', 'Wind Speed'], axis=1)
hotels_df['Hotel Name'] = ""
hotels_df.head()

,City,Latitude,Longitude,Country,Date,Hotel Name
13,Rome,34.2570,-85.1647,US,2021-06-17 17:57:41,
79,Hamilton,39.1834,-84.5333,US,2021-06-17 18:00:10,
99,Russell,32.3502,-85.1999,US,2021-06-17 18:00:06,
260,Prainha,-1.8000,-53.4800,BR,2021-06-17 18:01:38,
271,Havelock,34.8791,-76.9013,US,2021-06-17 18:01:43,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Create parameters function for json search requests
params = {
#     'location': 'city_geo',
    'types': 'lodging', # keyword
    'radius': 5000, 
    'key': key
}

# Iterate through hotels dataframe, get Lat & Lng, add location variable to params func
for i, row in hotels_df.iterrows():
    lat = row['Latitude']
    long = row['Longitude']
    params['location'] = f'{lat},{long}'

    # Construct url, initiate API request
    response = requests.get(base_url, params).json()
    json_results = response['results']

    try:
        # Add name to hotel dataframe
        hotels_df.loc[i, 'Hotel Name'] = json_results[0]['name']
        
    # If hotel information not available, show field and skip
    except:
        print('Missing field for city of {}...skipping'.format(row['City']))
        print('-'*26)
hotels_df.to_csv('./output_data/hotels_df.csv')
hotels_df.head()

,City,Latitude,Longitude,Country,Date,Hotel Name
13,Rome,34.2570,-85.1647,US,2021-06-17 17:57:41,La Quinta Inn & Suites by Wyndham Rome
79,Hamilton,39.1834,-84.5333,US,2021-06-17 18:00:10,Six Acres Bed & Breakfast
99,Russell,32.3502,-85.1999,US,2021-06-17 18:00:06,Macs
260,Prainha,-1.8000,-53.4800,BR,2021-06-17 18:01:38,Casa Rocha
271,Havelock,34.8791,-76.9013,US,2021-06-17 18:01:43,Sherwood Motel


### Add Hotel Markers

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotels_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))